## Setup

In [1]:
%cd ..

/Users/cesarchalco/Documents/projects/Machine-Learning-Engineering-with-Python-Second-Edition/Chapter03/practicing_sklearn_pipelines


/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
import sklearn as sk
import numpy as np

In [3]:
SEED = 42

In [4]:
TARGET = "Personality"

## Data collection

In [5]:
data = pd.read_csv("data/personality_dataset.csv")

In [6]:
data.sample(5, random_state=SEED)

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
141,10.000000,Yes,3.000000,3.0,Yes,5.0,3.0,Introvert
1557,4.505816,No,3.963354,5.0,No,14.0,5.0,Extrovert
485,5.000000,Yes,0.000000,0.0,Yes,0.0,2.0,Introvert
1712,10.000000,Yes,2.000000,2.0,Yes,2.0,1.0,Introvert
2250,7.000000,Yes,0.000000,0.0,Yes,2.0,1.0,Introvert


## Load and split

In [7]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    data.drop(columns=[TARGET]),
    data[TARGET],
    test_size=0.2,
    random_state=SEED
)

## Preprocess numeric and categorical features

In [8]:
numerical_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

In [9]:
X_train.shape, len(numerical_features), len(categorical_features)

((2320, 7), 5, 2)

In [10]:
numerical_transformer = sk.pipeline.Pipeline(
    steps=[
        ("imputer", sk.impute.SimpleImputer(strategy="median"))
    ]
)

In [11]:
categorical_transformer = sk.pipeline.Pipeline(
    steps=[
        ("encoder", sk.preprocessing.OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop="if_binary"))
    ]
)

## Assemble with `ColumnTransformer`

In [12]:
preprocessor = sk.compose.ColumnTransformer(
    transformers=[
        ("num_imputation", numerical_transformer, numerical_features),
        ("cat_encoding", categorical_transformer, categorical_features),
    ]
)

## Create a pipeline with the processor and a logistic regression

In [13]:
model = sk.linear_model.LogisticRegression(random_state=SEED)

In [14]:
clf = sk.pipeline.Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model_fitting", model)
    ]
)

## Pipeline testing

In [15]:
clf.fit(X_train, y_train)

/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_imputation',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('cat_encoding',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('model_fitting', LogisticRegression(random_state=42))])

In [18]:
predicted_probs_y_test = clf.predict_proba(X_test)[:, 1]

/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/practice-sklearn-pipes/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [19]:
pred_y_test = np.where(predicted_probs_y_test>=0.5, 1, 0)

## Evaluating pipeline

In [20]:
sk.metrics.roc_auc_score(
    y_true=y_test,
    y_score=predicted_probs_y_test
)

np.float64(0.9295762065844014)